# Análisis de sentimientos con reviews de productos de Amazon España (opcional)

Si has hecho ya el ejercicio de web scraping con `Requests` y `BeautifulSoup` habrás visto cómo extraer datos de una página web.

El dataset que utilizarás en este ejercicio (que no es obligatorio entregar) lo he generado utilizando `Scrapy` y `BeautifulSoup`, y contiene unas $700.000$ entradas con dos columnas: el número de estrellas dadas por un usuario a un determinado producto y el comentario sobre dicho producto; exactamente igual que en el ejercico de scraping.

Ahora, tu objetivo es utilizar técnicas de procesamiento de lenguaje natural para hacer un clasificador que sea capaz de distinguir (¡y predecir!) si un comentario es positivo o negativo.

Es un ejercicio MUY complicado, más que nada porque versa sobre técnicas que no hemos visto en clase. Así que si quieres resolverlo, te va a tocar estudiar y *buscar por tu cuenta*; exactamente igual que como sería en un puesto de trabajo. Dicho esto, daré un par de pistas:

+ El número de estrellas que un usuario da a un producto es el indicador de si a dicho usuario le ha gustado el producto o no. Una persona que da 5 estrellas (el máximo) a un producto probablemente esté contento con él, y el comentario será por tanto positivo; mientras que cuando una persona da 1 estrella a un producto es porque no está satisfecha... 
+ Teniendo el número de estrellas podríamos resolver el problema como si fuera de regresión; pero vamos a establecer una regla para convertirlo en problema de clasificación: *si una review tiene 4 o más estrellas, se trata de una review positiva; y será negativa si tiene menos de 4 estrellas*. Así que probablemente te toque transformar el número de estrellas en otra variable que sea *comentario positivo/negativo*.

Y... poco más. Lo más complicado será convertir el texto de cada review en algo que un clasificador pueda utilizar y entender (puesto que los modelos no entienden de palabras, sino de números). Aquí es donde te toca investigar las técnicas para hacerlo. El ejercicio se puede conseguir hacer, y obtener buenos resultados, utilizando únicamente Numpy, pandas y Scikit-Learn; pero siéntete libre de utilizar las bibliotecas que quieras.

Ahora escribiré una serie de *keywords* que probablemente te ayuden a saber qué buscar:

`bag of words, tokenizer, tf, idf, tf-idf, sklearn.feature_extraction, scipy.sparse, NLTK (opcional), stemmer, lemmatizer, stop-words removal, bigrams, trigrams`

No te desesperes si te encuentras muy perdido/a y no consigues sacar nada. Tras la fecha de entrega os daré un ejemplo de solución explicado con todo el detalle posible.

¡Ánimo y buena suerte!

# SOLUCIÓN 1
Se trata de un ejemplo de aprendizaje automático supervisado (clasificación) en el que las *features* son las palabras de cada comentario (*tokens*) y la *target* es el número de estrellas, que hay que convertir en variable binaria (positivo o negativo).  
Para resolverlo voy a utilizar herramientas de las librerías **nltk** y **sklearn.feature_extraction**. Lo más difícil es transformar el texto de los comentarios en un formato que pueda entender el clasificador Naïve Bayes que voy a usar.  
El primer paso es leer los datos. Como el fichero proporcionado viene en formato csv, voy a cargarlo con la función read_csv de pandas. Esto lo convertirá en un DataFrame que transformaré en array de numpy para manipularlo.

In [1]:
import pandas as pd

In [2]:
entrada=pd.read_csv("amazon_es_reviews.csv",sep=";")

Veo qué pinta tienen los datos y compruebo que son de tipo DataFrame.  
Los comentarios contienen faltas de ortografía, palabras mal escritas, signos de puntuación y mayúsculas.

In [3]:
entrada.head()

,comentario,estrellas
0,"Para chicas es perfecto, ya que la esfera no e...",4.0
1,Muy floja la cuerda y el anclaje es de mala ca...,1.0
2,"Razonablemente bien escrito, bien ambientado, ...",3.0
3,Hola! No suel o escribir muchas opiniones sobr...,5.0
4,A simple vista m parecia una buena camara pero...,1.0


In [69]:
type(entrada)

pandas.core.frame.DataFrame

A continuación separo los comentarios y el número de estrellas en sendos arrays de numpy para manipularlos independientemente. Más adelante la información aportada por cada uno se volverá a reunir.

In [83]:
import numpy as np

In [103]:
comentarios=np.array(entrada["comentario"])
puntuaciones=np.array(entrada["estrellas"])

In [139]:
print(len(comentarios))
print(len(puntuaciones))

702446
702446


In [162]:
print(type(comentarios))
print(type(puntuaciones))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Se trata de un problema de clasificación; así que voy a convertir la *target*, el número de estrellas (entre 1 y 5), en una variable nominal con dos valores (neg y pos) de manera que los valores de 4 y 5 estrellas los asigno al valor positivo y el resto de valores numéricos al valor negativo. Para ello defino una función de conversión:

In [135]:
def convertidor(numeros):
    listado=[]
    for numero in numeros:
        if numero>3:
            listado.append('pos')
            
        else:
            listado.append('neg')
            
    return listado

Hago unas pruebas con unos pocos números para comprobar que la función que he definido acepta y devuelve listas y funciona correctamente:

In [134]:
puntu=convertidor([1,2,0,4,5,3,0])
puntu

['neg', 'neg', 'neg', 'pos', 'pos', 'neg', 'neg']

In [137]:
puntuaciones[:10]

array([ 4.,  1.,  3.,  5.,  1.,  1.,  2.,  3.,  3.,  5.])

In [136]:
punt2=convertidor(puntuaciones[:10])
punt2

['pos', 'neg', 'neg', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos']

Parece que funciona; así que le paso ya como argumento todo el array de puntuaciones y compruebo su longitud y algunos de sus elementos:

In [138]:
punt=convertidor(puntuaciones)

In [140]:
len(punt)

702446

In [161]:
punt[:10]

['pos', 'neg', 'neg', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos']

Es correcto.  
Ya he transformado la columna del número de estrellas, ahora tengo que transformar la otra, la de los comentarios. Empiezo echándole un vistazo al array:

In [163]:
print(comentarios[:3])

print(type(comentarios))


[ 'Para chicas es perfecto, ya que la esfera no es muy grande y la correa se adapta a las muñecas más finas. Un pelín gordo (para mi gusto). La carga por movimiento no dura mucho. Después de 1-2 días sin llevarlo se para.'
 'Muy floja la cuerda y el anclaje es de mala calidad. El metal  Se dobla muy fácilmente. No lo recomiendo'
 'Razonablemente bien escrito, bien ambientado, quizás previsible, pero historia interesante. Los personajes están muy bien dibujados. Me parece un libro recomendable.']
<class 'numpy.ndarray'>


El siguiente paso (de los más importantes) es *tokenizar* todo el corpus de documentos de texto (cada comentario viene a ser un documento) para que el clasificador que se empleará posteriormente lo pueda entender.  
Los *tokens* son unidades de información en las que se descompone un texto (o imagen) a analizar, por ejemplo, palabras o letras, signos de puntuación, etc. En este caso voy a emplear un *tokenizer* para descomponer en palabras. Hay palabras que se repiten mucho en todos los textos y que no aportan ninguna información, como preposiciones, conjunciones, pronombres, etc. A estas palabras se las conoce como *stop words* y hay que eliminarlas del listado de *tokens*. Como estoy analizando textos en español, necesito una lista de *stop words* en español. Voy a importarla de la librería **nltk** (Natural Lenguage Tool Kit).
 

In [47]:
from nltk.corpus import stopwords

In [48]:
spanish_stopwords = stopwords.words('spanish')

In [54]:
type(spanish_stopwords)

list

In [61]:
spanish_stopwords[:15]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

Ya tengo la lista de *stop words* en español. Ahora, para *tokenizar* voy a usar la clase CountVectorizer de la librería **sklearn.feature_extraction**.

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

Hago una instancia de dicha clase pasándole la lista de *stop words* en español como atributo:

In [49]:
vectorizer_esp = CountVectorizer(min_df=1,stop_words=spanish_stopwords)

A la instancia que acabo de crear le aplico el método build_analyzer() y le paso una cadena de texto sencillo y un elemento de la lista de comentarios para comprobar que acepta/devuelve listas como entrada/salida y que *tokeniza* correctamente convirtiendo las palabras en minúsculas y eliminando signos de puntuación y las *stop words* incluidas en el listado anterior (este, un, de, es, para...):

In [50]:
analizador = vectorizer_esp.build_analyzer()

In [53]:
analizador("Este es un Documento de texTo para aNAlizar fácilmente")

['documento', 'texto', 'analizar', 'fácilmente']

In [116]:
tokens= analizador(comentarios[0])
print(tokens)
print(type(tokens))

['chicas', 'perfecto', 'esfera', 'grande', 'correa', 'adapta', 'muñecas', 'finas', 'pelín', 'gordo', 'gusto', 'carga', 'movimiento', 'dura', 'después', 'días', 'llevarlo']
<class 'list'>


Funciona correctamente. Ahora tengo que ir convirtiendo progresivamente el formato de los *tokens* hasta darles la estructura necesaria para poder usar el clasificador. Defino una función palabra_feat para convertir cada una de las listas de tokens de cada comentario en diccionarios en los que las claves son las palabras y los valores son todos el booleano True. Hago una pequeña prueba solo con la lista de *tokens* extraída del primer comentario.  

In [10]:
def palabra_feat(palabras):
    return dict([(palabra, True) for palabra in palabras])

In [112]:
prueba=palabra_feat(tokens)
prueba

{'adapta': True,
 'carga': True,
 'chicas': True,
 'correa': True,
 'después': True,
 'dura': True,
 'días': True,
 'esfera': True,
 'finas': True,
 'gordo': True,
 'grande': True,
 'gusto': True,
 'llevarlo': True,
 'movimiento': True,
 'muñecas': True,
 'pelín': True,
 'perfecto': True}

A continuación convierto todas las listas de *tokens* de todos los comentarios en diccionarios y vuelvo a recombinar esta información con la procedente del número de estrellas (recuérdese que esta ya está convertida en variable binaria de valores pos y neg). En este paso se obtiene el formato final que consiste en dos **listas** separadas (una para comentarios positivos y otra para negativos) de **duplas** donde cada dupla contiene un **diccionario** de *tokens* como claves y el booleano True como valores y la cadena 'neg' o 'pos' según sea negativo o positivo el comentario correspondiente, es decir:  
[ ( {'token1':True, 'token2':True,...},'neg'), ({...},'neg'),..., ({...},'neg') ]

In [152]:

negprueba=[]
posprueba=[]

for i in range(len(punt)):
    tokens0=analizador(comentarios[i])
    
    if punt[i]=="pos":
        posprueba.append((palabra_feat(tokens0),'pos'))
    if punt[i]=='neg':
        negprueba.append((palabra_feat(tokens0),'neg'))
    tokens0=[]

print(negprueba[:3])
print("")
print(posprueba[:3])
       

[({'fácilmente': True, 'dobla': True, 'cuerda': True, 'anclaje': True, 'calidad': True, 'metal': True, 'recomiendo': True, 'floja': True, 'mala': True}, 'neg'), ({'previsible': True, 'personajes': True, 'recomendable': True, 'ambientado': True, 'parece': True, 'historia': True, 'escrito': True, 'bien': True, 'razonablemente': True, 'interesante': True, 'dibujados': True, 'quizás': True, 'libro': True}, 'neg'), ({'entro': True, 'decepcione': True, 'casa': True, 'simple': True, 'parecia': True, 'baño': True, 'primer': True, 'camara': True, 'yeve': True, 'decepción': True, 'agua': True, 'probe': True, 'primero': True, 'gran': True, 'buena': True, 'acuario': True, 'adios': True, 'vista': True, 'muxo': True}, 'neg')]

[({'perfecto': True, 'esfera': True, 'gordo': True, 'grande': True, 'pelín': True, 'muñecas': True, 'después': True, 'llevarlo': True, 'adapta': True, 'finas': True, 'movimiento': True, 'días': True, 'carga': True, 'correa': True, 'dura': True, 'gusto': True, 'chicas': True}, 

Compruebo que la longitud de ambas listas suman los 702466 comentarios iniciales y calculo cuánto es el 75% de su extensión para dividir en conjuntos de *training* y *test*.

In [153]:
print(len(negprueba))
print(len(posprueba))

338711
363735


In [154]:
len(negprueba)*3/4

254033.25

In [155]:
len(posprueba)*3/4

272801.25

In [156]:
trainprueba=negprueba[:254033]+posprueba[:272801]
testprueba=negprueba[254033:]+posprueba[272801:]

In [157]:
print ("train on %d instances, test on %d instances" %(len(trainprueba), len(testprueba)))

train on 526834 instances, test on 175612 instances


Ahora importo el clasificador Naïve Bayes de la librería **nltk**.

In [4]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

Entreno el modelo:

In [158]:
clasificadorprueba = NaiveBayesClassifier.train(trainprueba)

Y uso el conjunto de *test* para obtener la puntuación del modelo:

In [159]:
print ('accuracy:', nltk.classify.util.accuracy(clasificadorprueba, testprueba))

accuracy: 0.799153816367902


Por último imprimo las palabras más relevantes con el método show_most_informative_features():

In [160]:
clasificadorprueba.show_most_informative_features()

Most Informative Features
                malisima = True              neg : pos    =    107.5 : 1.0
                malísima = True              neg : pos    =     58.0 : 1.0
                  fraude = True              neg : pos    =     56.0 : 1.0
              devolvería = True              neg : pos    =     55.6 : 1.0
                    timo = True              neg : pos    =     52.3 : 1.0
                 revisen = True              neg : pos    =     50.5 : 1.0
                devuelvo = True              neg : pos    =     49.9 : 1.0
                estafada = True              neg : pos    =     49.2 : 1.0
              desprendió = True              neg : pos    =     48.3 : 1.0
                  pesimo = True              neg : pos    =     45.2 : 1.0


# SOLUCIÓN 2
A continuación voy a resolver el ejercicio de manera distinta usando un algoritmo Random Forest y el modelo *bag of words* y construyendo un *tokenizer* propio para ir viendo cómo es el proceso paso a paso (el *tokenizer* de la solución 1 lo construí de automáticamente con la clase CountVectorizer).  
Empiezo importando la librería **re** para usar expresiones regulares y la clase BeautifulSoup de la librería **bs4**.


In [166]:
import re

In [169]:
from bs4 import BeautifulSoup

Hago un par de pruebas de limpieza de textos.

In [167]:
ejemplo="Esto es una Prueba 0 para reemplazar! ciertos , elementos;"

In [171]:
ejemplo1=BeautifulSoup(ejemplo)

C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Casy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [172]:
type(ejemplo1)

bs4.BeautifulSoup

In [173]:
solo_letras=re.sub("[^a-zA-Z]", # El patrón a buscar: todo lo que no sean letras.
                      " ",     # El patrón con el que reemplazar: espacio blanco.
                      ejemplo1.get_text() )
print(solo_letras)

Esto es una Prueba   para reemplazar  ciertos   elementos 


In [174]:
type(solo_letras)

str

El resultado devuelto es una cadena de caracteres en la que todos aquellos que no son letras han sido reemplazados por un espacio en blanco. Habrá que partir esa cadena para obtener palabras individuales. Esto se hace con el método split(). Además, hay mayúsculas que hay que transformar en minúsculas con el método lower(). El resultado devuelto ya es una lista de palabras individuales.

In [175]:
tokens1=solo_letras.lower().split()
print(tokens1)
print(type(tokens1))

['esto', 'es', 'una', 'prueba', 'para', 'reemplazar', 'ciertos', 'elementos']
<class 'list'>


Ahora toca eliminar las *stop words*. El resultado sigue siendo una lista.

In [177]:
palabras = [palabra for palabra in tokens1 if not palabra in spanish_stopwords]
print(palabras)
print(type(palabras))

['prueba', 'reemplazar', 'ciertos', 'elementos']
<class 'list'>


Una vez comprobados los pasos de *tokenización* los agrupo todos en la función mi_tokenizer de forma que esté todo ordenado y sea más sencillo de usar.

In [199]:
def mi_tokenizer( texto ):
    # Función para convertir texto en bruto en una cadena de palabras significativas.
    # La entrada es una sola cadena (cada uno de los comentarios de los clientes).
    # La salida es una sola cadena con las palabras de la crítica.
    
    # 1. Se limpia el texto de código HTML si lo hubiera.
    texto_limpio = BeautifulSoup(texto).get_text() 
    
    # 2. Se eliminan todos aquellos caracteres que no sean letras (incluye tildes).        
    solo_letras = re.sub("[^a-zA-ZñÑáéíóúüÁÉÍÓÚÜ]", " ", texto_limpio) 

    # 3. Se convierte en minúsculas y se divide en palabras individuales.
    solo_palabras = solo_letras.lower().split()                             
    
    # 4. Convierto la lista de stop words en un set porque en Python es mucho más
    #   rápido buscar en un set que en una lista y se va a usar miles de veces.
    stops = set(stopwords.words("spanish"))                  
 
    # 5. Se eliminan las stop words.
    palabras_significativas=[palabra for palabra in solo_palabras if not palabra in stops]   

    # 6. Se juntan de nuevo en una cadena las palabras separadas por espacios.
    
    return( " ".join( palabras_significativas ))
 

Hago un par de pruebas de tokenizar un solo elemento de los comentarios y luego dos para comprobar que se devuelve una lista de cadenas, una para cada comentario.  
Veo que las palabras devueltas son exactamente las mismas 17 que en la solución 1. Para el segundo comentario devuelve 9 palabras.

In [200]:
limpio=mi_tokenizer(comentarios[0])
print(limpio)
print(type(limpio))

chicas perfecto esfera grande correa adapta muñecas finas pelín gordo gusto carga movimiento dura después días llevarlo
<class 'str'>


C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Casy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [201]:
limpio1=[]

for i in range(2):
    limpio1.append(mi_tokenizer(comentarios[i]))

print(limpio1)
print(type(limpio1))

['chicas perfecto esfera grande correa adapta muñecas finas pelín gordo gusto carga movimiento dura después días llevarlo', 'floja cuerda anclaje mala calidad metal dobla fácilmente recomiendo']
<class 'list'>


C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Casy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Funciona correctamente. Ya puedo generar los subconjuntos de entrenamiento (75%) y test (25%).

In [209]:
train=comentarios[:526834]
test=comentarios[526834:]
print(len(train))
print(len(test))
print(type(train))
print(type(test))

526834
175612
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [213]:
limpio_train=[]

for i in range(len(train)):
    if( (i+1)%2000 == 0 ):
        print ("Comentario %d de %d\n procesado." % ( i+1, len(train) ))
    limpio_train.append(mi_tokenizer(train[i]))

print(limpio_train[:2])
print(type(limpio_train))

C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Casy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Comentario 2000 de 526834
 procesado.
Comentario 4000 de 526834
 procesado.
Comentario 6000 de 526834
 procesado.
Comentario 8000 de 526834
 procesado.
Comentario 10000 de 526834
 procesado.
Comentario 12000 de 526834
 procesado.
Comentario 14000 de 526834
 procesado.
Comentario 16000 de 526834
 procesado.
Comentario 18000 de 526834
 procesado.
Comentario 20000 de 526834
 procesado.
Comentario 22000 de 526834
 procesado.
Comentario 24000 de 526834
 procesado.
Comentario 26000 de 526834
 procesado.


C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.......... ............. ............. ......... ....... ...... ......... ............. ........ ..... ........ ...... ...... ...... .......... ........... .......... .......... ....... .....'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 28000 de 526834
 procesado.
Comentario 30000 de 526834
 procesado.
Comentario 32000 de 526834
 procesado.
Comentario 34000 de 526834
 procesado.
Comentario 36000 de 526834
 procesado.
Comentario 38000 de 526834
 procesado.
Comentario 40000 de 526834
 procesado.
Comentario 42000 de 526834
 procesado.
Comentario 44000 de 526834
 procesado.
Comentario 46000 de 526834
 procesado.
Comentario 48000 de 526834
 procesado.
Comentario 50000 de 526834
 procesado.
Comentario 52000 de 526834
 procesado.
Comentario 54000 de 526834
 procesado.
Comentario 56000 de 526834
 procesado.
Comentario 58000 de 526834
 procesado.
Comentario 60000 de 526834
 procesado.
Comentario 62000 de 526834
 procesado.
Comentario 64000 de 526834
 procesado.
Comentario 66000 de 526834
 procesado.
Comentario 68000 de 526834
 procesado.
Comentario 70000 de 526834
 procesado.
Comentario 72000 de 526834
 procesado.
Comentario 74000 de 526834
 procesado.
Comentario 76000 de 526834
 procesado.
Comentario 78000 de 52683

C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'..... ... .. . .... . ....... ....... ... ..... ......... ............ ..... . ..... .... . ..... ... .... ... . ...... .'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 434000 de 526834
 procesado.
Comentario 436000 de 526834
 procesado.
Comentario 438000 de 526834
 procesado.
Comentario 440000 de 526834
 procesado.
Comentario 442000 de 526834
 procesado.
Comentario 444000 de 526834
 procesado.
Comentario 446000 de 526834
 procesado.
Comentario 448000 de 526834
 procesado.
Comentario 450000 de 526834
 procesado.
Comentario 452000 de 526834
 procesado.
Comentario 454000 de 526834
 procesado.
Comentario 456000 de 526834
 procesado.
Comentario 458000 de 526834
 procesado.
Comentario 460000 de 526834
 procesado.
Comentario 462000 de 526834
 procesado.
Comentario 464000 de 526834
 procesado.
Comentario 466000 de 526834
 procesado.
Comentario 468000 de 526834
 procesado.
Comentario 470000 de 526834
 procesado.
Comentario 472000 de 526834
 procesado.
Comentario 474000 de 526834
 procesado.
Comentario 476000 de 526834
 procesado.
Comentario 478000 de 526834
 procesado.
Comentario 480000 de 526834
 procesado.
Comentario 482000 de 526834
 procesado.


C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'............. .................... ................ .............. .............. ........... . .... ..... .... .... ... ... ... ... ... ... ... ... ..'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 508000 de 526834
 procesado.
Comentario 510000 de 526834
 procesado.
Comentario 512000 de 526834
 procesado.
Comentario 514000 de 526834
 procesado.
Comentario 516000 de 526834
 procesado.
Comentario 518000 de 526834
 procesado.
Comentario 520000 de 526834
 procesado.
Comentario 522000 de 526834
 procesado.
Comentario 524000 de 526834
 procesado.
Comentario 526000 de 526834
 procesado.
['chicas perfecto esfera grande correa adapta muñecas finas pelín gordo gusto carga movimiento dura después días llevarlo', 'floja cuerda anclaje mala calidad metal dobla fácilmente recomiendo']
<class 'list'>


Este paso le llevó al PC unas 12 horas de cómputo.  
A continuación genero una instancia de la clase CountVectorizer sin apenas pasarle atributos puesto que el proceso de *tokenize* ya lo he hecho de manera personalizada.

In [197]:
vectorizer2 = CountVectorizer(analyzer = "word",  
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None)
                        

Hago una prueba aplicando el método fit_transform() a la lista de prueba de dos elementos con sendas cadenas que contienen las palabras de los dos primeros comentarios.  
Este método transforma las cadenas de palabras en una matriz dispersa (*sparse matrix*) con 26 elementos, correspondientes a 26 palabras. Esto es correcto puesto que, como se acaba de ver unas celdas más arriba, el primer comentario produce 17 palabras y el segundo 9. Cada fila de la matriz corresponde a un comentario (documento de texto) y cada columna a una palabra distinta. En cada celda de la matriz se coloca la cantidad de veces que una determinada palabra aparece en un determinado comentario.  
Una matriz dispersa es una matriz grande en la que la mayoría de sus términos son nulos. Aquí aparecen las matrices dispersas porque la mayoría de las pocas palabras de un determinado comentario no aparecen en los demás. Por tanto, en las celdas correspondientes habrá ceros. Como son muchísimos comentarios, habrá muchísimos ceros. Esta información ocuparía mucha memoria. Por tanto, lo que se hace es almacenar solo los términos no nulos de la matriz en un formato compacto, CSR (Compressed Sparse Row format).

In [206]:
bag_prueba = vectorizer2.fit_transform(limpio1)
bag_prueba

<2x26 sparse matrix of type '<class 'numpy.int64'>'
	with 26 stored elements in Compressed Sparse Row format>

Una vez comprobado que funciona, aplico el método fit_transform() a la lista con todos los comentarios del conjunto de entrenamiento. Este método sirve para dos cosas: para ajustar el modelo y para generar la matriz dispersa del *bag of words*. 

In [214]:
bag_of_words = vectorizer2.fit_transform(limpio_train)
bag_of_words

<526834x224646 sparse matrix of type '<class 'numpy.int64'>'
	with 11316265 stored elements in Compressed Sparse Row format>

La matriz dispersa consta de 526834 filas, correspondientes a los 526834 comentarios del conjunto de entrenamiento, y 224646 columnas, correspondientes a las 224646 palabras significativas del corpus de comentarios, es decir, el *bag of words* está formado por 224646 palabras.  
Puedo mirar el vocabulario del *bag of words* del conjunto de entrenamiento.

In [217]:
vocabulario = vectorizer2.get_feature_names()
print(type(vocabulario))

<class 'list'>


In [241]:
vocabulario[110030:110050]

['impulsivos',
 'impulso',
 'impulsor',
 'impulsora',
 'impulsos',
 'impulsándola',
 'impulsé',
 'impulsó',
 'impune',
 'impunemente',
 'impunes',
 'impunidad',
 'impuntuales',
 'impuntualidad',
 'impura',
 'impureza',
 'impurezas',
 'impuridades',
 'impuro',
 'impuros']

Se ve que hay palabras muy parecidas con el mismo lexema. Podría ser útil llevar a cabo un proceso de stemming para considerar todas las pertenecientes a la misma familia como una sola.  
Paso ahora a crear un modelo Random Forest.

In [242]:
from sklearn.ensemble import RandomForestClassifier

Genero una instancia de Random Forests con 100 árboles.

In [255]:
bosque = RandomForestClassifier(n_estimators = 30)

Ahora entreno el modelo con las palabras del *bag of words* como *features* y los valores del sentimiento (pos o neg) como *target*. Este proceso duró unas 9 horas.

In [256]:
bosque_entrenado = bosque.fit( bag_of_words, punt[:526834] )

In [257]:
bosque_entrenado

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

Por último, queda aplicar el modelo entrenado al conjunto de *test*. Antes hay que transformar este conjunto de la misma forma que se hizo con el de entrenamiento. El proceso requirió unas 4 horas (claro, un tercio del tiempo para el de entrenamiento). 

In [259]:
limpio_test=[]

for i in range(len(test)):
    if( (i+1)%2000 == 0 ):
        print ("Comentario %d de %d procesado.\n" % ( i+1, len(test) ))
    limpio_test.append(mi_tokenizer(test[i]))

print(limpio_test[:2])
print(type(limpio_test))

C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Casy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Comentario 2000 de 175612 procesado.

Comentario 4000 de 175612 procesado.

Comentario 6000 de 175612 procesado.

Comentario 8000 de 175612 procesado.

Comentario 10000 de 175612 procesado.

Comentario 12000 de 175612 procesado.

Comentario 14000 de 175612 procesado.

Comentario 16000 de 175612 procesado.

Comentario 18000 de 175612 procesado.

Comentario 20000 de 175612 procesado.

Comentario 22000 de 175612 procesado.



C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'........... .............. .............. ............. ............. ............... ............. .......... ........ . . ............ . ............. . . ...... ........ ...... ........'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 24000 de 175612 procesado.

Comentario 26000 de 175612 procesado.

Comentario 28000 de 175612 procesado.

Comentario 30000 de 175612 procesado.

Comentario 32000 de 175612 procesado.



C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 34000 de 175612 procesado.

Comentario 36000 de 175612 procesado.

Comentario 38000 de 175612 procesado.

Comentario 40000 de 175612 procesado.

Comentario 42000 de 175612 procesado.

Comentario 44000 de 175612 procesado.

Comentario 46000 de 175612 procesado.

Comentario 48000 de 175612 procesado.

Comentario 50000 de 175612 procesado.

Comentario 52000 de 175612 procesado.



C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'....... ........ ...... .. .. . ......... ....... ....... ...... .. ... ...... ..... ........ ...... ...... .... ........ .'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 54000 de 175612 procesado.

Comentario 56000 de 175612 procesado.

Comentario 58000 de 175612 procesado.

Comentario 60000 de 175612 procesado.

Comentario 62000 de 175612 procesado.

Comentario 64000 de 175612 procesado.

Comentario 66000 de 175612 procesado.

Comentario 68000 de 175612 procesado.

Comentario 70000 de 175612 procesado.

Comentario 72000 de 175612 procesado.

Comentario 74000 de 175612 procesado.

Comentario 76000 de 175612 procesado.

Comentario 78000 de 175612 procesado.

Comentario 80000 de 175612 procesado.

Comentario 82000 de 175612 procesado.

Comentario 84000 de 175612 procesado.

Comentario 86000 de 175612 procesado.

Comentario 88000 de 175612 procesado.

Comentario 90000 de 175612 procesado.

Comentario 92000 de 175612 procesado.

Comentario 94000 de 175612 procesado.

Comentario 96000 de 175612 procesado.

Comentario 98000 de 175612 procesado.

Comentario 100000 de 175612 procesado.

Comentario 102000 de 175612 procesado.

Comentario 104000 de 17

C:\Users\Casy\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.......... ............ ............ ............... .............. ............... ........... ............ ............ ...... ........... ..  . .. ....  ..... .... .... .... ...  .....'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Comentario 166000 de 175612 procesado.

Comentario 168000 de 175612 procesado.

Comentario 170000 de 175612 procesado.

Comentario 172000 de 175612 procesado.

Comentario 174000 de 175612 procesado.

['aunque costo esperaba mejor instalación software manejo sencillo pesar venir inglés calidad imagen sacar fotos grabar vídeos escasa', 'pueden ver películas si conecto disco duro tv visto comentarios necesita ser multimedia así alguien probado']
<class 'list'>


Del mismo modo, genero el *bag of words* del conjunto de test. Ahora, en cambio, no se le aplica el método fit_transform() sino transform() porque aquel era para ajustar el modelo con el conjunto de entrenamiento (y obtener la matriz dispersa correspondiente) y ahora solo se necesita obtener la representación de *bag of words* con la matriz dispersa del conjunto de test.

In [261]:
bag_of_words_test = vectorizer2.transform(limpio_test)

In [262]:
bag_of_words_test

<175612x224646 sparse matrix of type '<class 'numpy.int64'>'
	with 3743352 stored elements in Compressed Sparse Row format>

In [263]:
resultado=bosque_entrenado.predict(bag_of_words_test)
print(type(resultado))

<class 'numpy.ndarray'>


In [264]:
resultado[:15]

array(['neg', 'neg', 'pos', 'pos', 'neg', 'neg', 'neg', 'pos', 'neg',
       'neg', 'pos', 'pos', 'neg', 'pos', 'pos'], 
      dtype='<U3')

In [265]:
bosque_entrenado.score(bag_of_words_test,punt[526834:])

0.79147780333917961

Como puede apreciarse, la precisión de este modelo con Random Forest es algo inferior al de la solución 1 con Naïve Bayes. Además, este modelo ha tardado unas 25 veces más en completar su ejecución. Nótese que la otra solución trabajaba con diccionarios, mientras que esta lo hacía con listas. Y es mucho más rápido buscar dentro de un set o un diccionario que en una lista.

Podría conseguirse mejor precisión realizando un proceso de stemming previo para unificar todas las palabras con el mismo lexema. El uso de bigrams también podría incrementar la tasa de aciertos. Pero quizá sea más útil en un idioma como el inglés, donde existen muchas palabras compuestas, que en uno como el español, en que no es tan habitual la construcción de palabras mediante composición.